# B1 PREPROCESSING

In [31]:
import pandas as pd
import openpyxl
import numpy as np

## Import Dataset from excel file. 

In [13]:
initial_df=pd.read_excel('data_akbilgic.xlsx',header=1)

C:\Users\marko\anaconda3\envs\da_project\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [14]:
initial_df.head()

,date,ISE,ISE.1,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
0,2009-01-05,0.035754,0.038376,-0.004679,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524
1,2009-01-06,0.025426,0.031813,0.007787,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773
2,2009-01-07,-0.028862,-0.026353,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015
3,2009-01-08,-0.062208,-0.084716,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424
4,2009-01-09,0.009860,0.009658,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802


In [15]:
len(initial_df) #length of the initial dataset.

536

## Preprocessing function

In [16]:
def data_preprocessing(df):
    df=df.loc[0:529] #keep the rows from 0 to 529 (first 530 rows)
    df=df.drop(columns=['date']) #drop date column date
    #df.to_csv('./data_akbilgic_clean.csv', header=True, index=False ,sep=';') #save the cleansed data in a csv file.
    return df

In [17]:
preprocessed_df=data_preprocessing(initial_df)

In [18]:
preprocessed_df.head()

,ISE,ISE.1,SP,DAX,FTSE,NIKKEI,BOVESPA,EU,EM
0,0.035754,0.038376,-0.004679,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524
1,0.025426,0.031813,0.007787,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773
2,-0.028862,-0.026353,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015
3,-0.062208,-0.084716,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424
4,0.009860,0.009658,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802


In [19]:
len(preprocessed_df)

530

## Make the dataset supervised 

In [20]:
def timeseries_to_supervised(df, n_in, n_out):
   agg = pd.DataFrame()

   for i in range(n_in, 0, -1):
      df_shifted = df.shift(i).copy()
      df_shifted.rename(columns=lambda x: ('%s(t-%d)' % (x, i)), inplace=True)
      agg = pd.concat([agg, df_shifted], axis=1)

   for i in range(0, n_out):
      df_shifted = df.shift(-i).copy()
      if i == 0:
         df_shifted.rename(columns=lambda x: ('%s(t)' % (x)), inplace=True)
      else:
         df_shifted.rename(columns=lambda x: ('%s(t+%d)' % (x, i)), inplace=True)
      agg = pd.concat([agg, df_shifted], axis=1)
   agg.dropna(inplace=True)
   return agg

In [21]:
supervised_df=timeseries_to_supervised(preprocessed_df,4,1)# timeseries consists of 4 input steps.

In [22]:
supervised_df.head()

,ISE(t-4),ISE.1(t-4),SP(t-4),DAX(t-4),FTSE(t-4),NIKKEI(t-4),BOVESPA(t-4),EU(t-4),EM(t-4),ISE(t-3),...,EM(t-1),ISE(t),ISE.1(t),SP(t),DAX(t),FTSE(t),NIKKEI(t),BOVESPA(t),EU(t),EM(t)
4,0.035754,0.038376,-0.004679,0.002193,0.003894,0.000000,0.031190,0.012698,0.028524,0.025426,...,-0.019424,0.009860,0.009658,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802
5,0.025426,0.031813,0.007787,0.008455,0.012866,0.004162,0.018920,0.011341,0.008773,-0.028862,...,-0.007802,-0.029191,-0.042361,-0.022823,-0.013526,-0.005026,-0.049039,-0.053849,-0.012451,-0.022630
6,-0.028862,-0.026353,-0.030469,-0.017833,-0.028735,0.017293,-0.035899,-0.017073,-0.020015,-0.062208,...,-0.022630,0.015445,-0.000272,0.001757,-0.017674,-0.006141,0.000000,0.003572,-0.012220,-0.004827
7,-0.062208,-0.084716,0.003391,-0.011726,-0.000466,-0.040061,0.028283,-0.005561,-0.019424,0.009860,...,-0.004827,-0.041168,-0.035552,-0.034032,-0.047383,-0.050945,0.002912,-0.040302,-0.045220,-0.008677
8,0.009860,0.009658,-0.021533,-0.019873,-0.012710,-0.004474,-0.009764,-0.010989,-0.007802,-0.029191,...,-0.008677,0.000662,-0.017268,0.001328,-0.019551,-0.014335,-0.050448,0.030314,-0.012070,-0.023429


In [23]:
supervised_df.keys()

Index(['ISE(t-4)', 'ISE.1(t-4)', 'SP(t-4)', 'DAX(t-4)', 'FTSE(t-4)',
       'NIKKEI(t-4)', 'BOVESPA(t-4)', 'EU(t-4)', 'EM(t-4)', 'ISE(t-3)',
       'ISE.1(t-3)', 'SP(t-3)', 'DAX(t-3)', 'FTSE(t-3)', 'NIKKEI(t-3)',
       'BOVESPA(t-3)', 'EU(t-3)', 'EM(t-3)', 'ISE(t-2)', 'ISE.1(t-2)',
       'SP(t-2)', 'DAX(t-2)', 'FTSE(t-2)', 'NIKKEI(t-2)', 'BOVESPA(t-2)',
       'EU(t-2)', 'EM(t-2)', 'ISE(t-1)', 'ISE.1(t-1)', 'SP(t-1)', 'DAX(t-1)',
       'FTSE(t-1)', 'NIKKEI(t-1)', 'BOVESPA(t-1)', 'EU(t-1)', 'EM(t-1)',
       'ISE(t)', 'ISE.1(t)', 'SP(t)', 'DAX(t)', 'FTSE(t)', 'NIKKEI(t)',
       'BOVESPA(t)', 'EU(t)', 'EM(t)'],
      dtype='object')

## Give  [samples, steps, features] shape.

In [26]:
X, y = supervised_df[[('ISE(t-%d)' % i) for i in range(4, 0, -1)]].values, supervised_df['ISE(t)'].values
print(X.shape, y.shape)

(526, 4) (526,)


In [28]:
samples = supervised_df.shape[0]
steps = 1
print("Samples",samples)
print("Steps",steps)
features_in = 4
features_out = 1

Samples 526
Steps 1


In [39]:
X = np.reshape(X, (samples, steps, features_in))# (Samples,steps,input_features)=(263,1,4)
y = np.reshape(y, (samples, steps, features_out))

In [40]:
print("Data input shape:",X.shape,"Data output shape:", y.shape)

Data input shape: (526, 1, 4) Data output shape: (526, 1, 1)


### Save the supervised data to a csv file.

In [29]:
#supervised_df.to_csv('./data_akbilgic_supervised.csv', header=True, index=False ,sep=';')